In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Loading the datasets in chunks

In [10]:
file_path = "../datasets/processed_logs.csv"
chunk_size = 500000

In [13]:
chunks = pd.read_csv(file_path, chunksize=chunk_size)
logs_df = pd.concat(chunks)

/var/folders/hw/2qqt9k2547j0gf2k4f_hq0lc0000gn/T/ipykernel_29072/492501328.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  logs_df = pd.concat(chunks)


In [7]:
logs_df.tail()

,timestamp,level,message,source
499995,2005-06-14 10:57:15.124924,FATAL,guaranteed instruction cache block touch.0,BGL
499996,2005-06-14 10:57:15.152929,FATAL,guaranteed instruction cache block touch.0,BGL
499997,2005-06-14 10:57:15.162961,FATAL,program interrupt: fp compare...............0,BGL
499998,2005-06-14 10:57:15.174778,FATAL,guaranteed instruction cache block touch.0,BGL
499999,2005-06-14 10:57:15.184791,FATAL,program interrupt: fp compare...............0,BGL


Exploratory Data Analysis

In [16]:
# print(logs_df.info())
print(logs_df['level'].value_counts())

level
INFO                                5954567
WARN                                 833297
ERROR                                625355
CRITICAL                             624647
FATAL                                321471
                                     ...   
GoogleSoftwareUpdateAgent[35089]          1
netbiosd[35901]                           1
netbiosd[31279]                           1
helpd[36107]                              1
GoogleSoftwareUpdateAgent[33940]          1
Name: count, Length: 1040, dtype: int64
